# Digital Skills EU – Gap-Analyse 2021 → 2023
Codecademy ML/AI Foundations • Portfolio Project  
*Datensatz:* Eurostat ISOC_SK_DSKL_I21 • **BASIC digital skills, 16-74 J., % Individuals**


In [1]:
import pandas as pd
from pathlib import Path

# ----------------------------------------
# Projektpfad – exakt wie auf deinem PC
# ----------------------------------------
ROOT = Path(r"C:\Users\Andreas Ehof\OneDrive\Persöhnliche Entwicklung\KI"
            r"\Codecademy_ML_AI_Engineer\7. Portfolio Project_ML,AI Foundations Skill Path"
            r"\digital-skills-eu")

csv_path = ROOT / "data" / "isoc_sk_dskl_i21.csv"     # ggf. .csv.gz -> compression="gzip"

# Eurostat SDMX-CSV 1.0: erste Zeile = Kommentar → skiprows=1
wide = pd.read_csv(csv_path, skiprows=1)

# Länder-Spalte umbenennen
wide = wide.rename(columns={wide.columns[0]: "geo"})
wide.head()


,geo,17/12/24 11:00:00,Annual,All individuals,Individuals with above basic overall digital skills (all five component indicators are at above basic level),Percentage of individuals,Albania,2021,4.01,Unnamed: 9,Unnamed: 10
0,ESTAT:ISOC_SK_DSKL_I21$DEFAULTVIEW(1.0),17/12/24 11:00:00,Annual,All individuals,Individuals with above basic overall digital s...,Percentage of individuals,Albania,2023,7.77,NaN,NaN
1,ESTAT:ISOC_SK_DSKL_I21$DEFAULTVIEW(1.0),17/12/24 11:00:00,Annual,All individuals,Individuals with above basic overall digital s...,Percentage of individuals,Austria,2021,33.28,NaN,NaN
2,ESTAT:ISOC_SK_DSKL_I21$DEFAULTVIEW(1.0),17/12/24 11:00:00,Annual,All individuals,Individuals with above basic overall digital s...,Percentage of individuals,Austria,2023,32.04,NaN,NaN
3,ESTAT:ISOC_SK_DSKL_I21$DEFAULTVIEW(1.0),17/12/24 11:00:00,Annual,All individuals,Individuals with above basic overall digital s...,Percentage of individuals,Bosnia and Herzegovina,2021,5.35,NaN,NaN
4,ESTAT:ISOC_SK_DSKL_I21$DEFAULTVIEW(1.0),17/12/24 11:00:00,Annual,All individuals,Individuals with above basic overall digital s...,Percentage of individuals,Bosnia and Herzegovina,2023,6.88,NaN,NaN


## 2  Daten in Long-Format bringen  
Nur Jahres-Spalten behalten, feste Zusatz-Dimensionen ergänzen.


In [2]:
# Wide ➜ Long
skills = wide.melt(id_vars="geo", var_name="year", value_name="values")

# feste Dimensionen
skills["digski"] = "BASIC"
skills["sex"]    = "T"
skills["age"]    = "Y16-74"
skills["unit"]   = "PC_IND"

# nur echte Jahreswerte (vier Ziffern)
skills = skills[skills["year"].str.fullmatch(r"\d{4}")]

# Datentypen
skills["year"]   = skills["year"].astype(int)
skills["values"] = pd.to_numeric(skills["values"], errors="coerce")

print(skills.shape)
skills.head()


(3475, 7)


,geo,year,values,digski,sex,age,unit
20850,ESTAT:ISOC_SK_DSKL_I21$DEFAULTVIEW(1.0),2021,2023,BASIC,T,Y16-74,PC_IND
20851,ESTAT:ISOC_SK_DSKL_I21$DEFAULTVIEW(1.0),2021,2021,BASIC,T,Y16-74,PC_IND
20852,ESTAT:ISOC_SK_DSKL_I21$DEFAULTVIEW(1.0),2021,2023,BASIC,T,Y16-74,PC_IND
20853,ESTAT:ISOC_SK_DSKL_I21$DEFAULTVIEW(1.0),2021,2021,BASIC,T,Y16-74,PC_IND
20854,ESTAT:ISOC_SK_DSKL_I21$DEFAULTVIEW(1.0),2021,2023,BASIC,T,Y16-74,PC_IND


## 3  Top- & Bottom-Länder 2023

In [3]:
rank23 = (
    skills[skills["year"] == 2023]
    .sort_values("values", ascending=False)
    .reset_index(drop=True)
)
rank23.head(10)


,geo,year,values,digski,sex,age,unit


## 4  Δ-Analyse 2021 → 2023  
Differenz pro Land, gruppiert nach Gewinnern & Nachzüglern.


In [4]:
delta = (
    skills.pivot(index="geo", columns="year", values="values")
    .assign(delta_21_23=lambda x: x[2023] - x[2021])
    .sort_values("delta_21_23", ascending=False)
)
delta.head()


ValueError: Index contains duplicate entries, cannot reshape